# Sisteme Multi-Agent: Dilema prizonierului
 - Andrei Olaru
 - Tudor Berariu
 


#### Scopul laboratorului

Scopul acestui laborator este de a vă familiariza cu noțiunea de [agent software](https://en.wikipedia.org/wiki/Software_agent). O parte din preocupările din domeniul sistemelor multi-agent se axează pe probleme din teoria jocurilor și modul cum agenții pot juca jocuri teoretice cât mai bine.

#### Dilema prizonierului

Dilema prizonierului (vezi și [aici](https://en.wikipedia.org/wiki/Prisoner's_dilemma)) este un joc de doi jucători în care fiecare jucător are la dispoziție două acțiuni: cooperarea cu celălalt jucător sau trădarea acestuia. În funcție de acțiunile alese, fiecare dintre cei doi jucători primește o recompensă. Recompensa agregată maximă este când ambii jucători cooperează, dar atracția este mai mare către trădare (vezi [matricea de joc](https://en.wikipedia.org/wiki/Prisoner%27s_dilemma#Generalized_form)).

#### Dilema prizonierului, iterată

Dacă pentru un singur joc trădarea este opțiunea evidentă (cf. Echilibrului Nash), strategiile pot deveni mai complexe atunci când se joacă mai multe jocuri între aceiași doi jucători (poate exista ideea unei cooperări, pentru a crește scorul ambilor agenți).

#### Cerință

Se cere implementarea a 4 strategii 'standard' (vezi document pdf) pentru varianta iterată a jocului, împreună cu o **strategie proprie** care să se comporte acceptabil împotriva celorlalte, într-un turneu. Ceea ce ne interesează este **scorul** total obținut, mai ales raportat la numărul de jocuri (score/games).

In [37]:
from random import choice, randint

D = 'Defect'
C = 'Cooperate'

rewards = {(C, C): (3, 3), (C, D): (0, 5), (D, C): (5, 0), (D, D): (2, 2)}

In [38]:
# O strategie de joc este caracterizată de o funcție care întoarce, la fiecare apel, acțiunea aleasă de strategie.
# Parametrul primit de funcție este o listă de tupluri care conține acțiunile jucate anterior în cursul aceluiași 
# joc iterat.
# Fiecare tuplu din listă corespunde unui joc individual de dilema prizonierului și conține pe prima poziție 
# acțiunea aleasă de acest agent și pe a doua acțiunea jucată de oponent.

def AllD(_):
    return D

def Random(_):
    # TODO
    return choice([D, C])

def TFT(information):
    if len(information) == 0:
        return C
    
    return information[-1][1]

def Joss(information):
    TFT_action = TFT(information)
    
    if TFT_action == D:
        return D
    
    return choice([D, C, C, C, C, C, C, C, C, C])

index = 0
def CCD():
    global index
    
    if index == 0 or index == 1:
        index += 1
        return C
    
    index = 0
    return D

tester_strategy = -1
def Tester(information):
    global tester_strategy
    
    if len(information) == 0:
        return D
    
    if len(information) == 1:
        if information[0][1] == D:
            tester_strategy = 0
            return TFT(information)
        
        tester_strategy = 1
        return CCD()
    
    if tester_strategy == 0:
        return TFT(information)
    else:
        return CCD()

# TODO: o nouă strategie
def my_strategy(information):
    if len(information) == 0:
        return D
    
    # culeg toate actiunile adversarului
    inf2 = [inf[1] for inf in information]
    
    # tit for tat random pe toate actiunile lui, nu doar pe ultima
    return choice(inf2)
    

In [39]:
# TODO de activat strategiile aici
availableStrategies = [
    ('All-D', AllD),
    ('Random', Random),
    ('Tit-For-Tat', TFT), 
    ('Joss', Joss),
    ('Tester', Tester),
    ('MyStrategy', my_strategy),
]

In [40]:
strategies = []
for (name, proc) in availableStrategies:
    strategies.append({'name': name, 'procedure': proc, 'wins': 0, 'score': 0, 'games': 0, 'plays': {}})

# joacă un joc între A și B, întoarce recompensele asociate
def play_game(players, verbose = False):
    choices = [p['strategy']['procedure'](p['information']) for p in players]
    for i in range(2):
        players[i]['information'].append((choices[i], choices[1 - i]))
    if verbose: print(players[0]['strategy']['name']+" vs "+players[1]['strategy']['name']+" choices: "+str(choices)+" rewards: "+str(rewards[tuple(choices)]))
    return rewards[tuple(choices)]
    
# joacă `iterations` jocuri între A și B, întorcând scorul asociat întregului joc iterat
def play_iterated_pd(players, n_iterations, verbose = False):
    score = (0, 0)
    for i in range(n_iterations):
        rewardsAB = play_game(players, verbose)
        score = tuple([score[pi] + rewardsAB[pi] for pi in range(2)])
    if verbose: print("== result: "+str(score))
    return score

# joacă un turneu de n jocuri de câte n iterații, alegând aleator între strategiile date în `strategies`
def tournament(n_games, n_iterations, strategies, verbose = False):
    for game in range(n_games):
        agents = []
        strat = []
        for i in range(2):
            agents.append({'strategy': choice(strategies), 'information': []})
            strat.append(agents[i]['strategy'])
        for i in range(2):
            for j in range(2):
                if i != j:
                    if strat[j]['name'] not in strat[i]['plays']:
                        strat[i]['plays'][strat[j]['name']] = 1
                    else:
                        strat[i]['plays'][strat[j]['name']] += 1
        scores = play_iterated_pd(agents, n_iterations, verbose)
        result = (0, 0)
        if scores[0] > scores[1]:
            result = (1, 0)
        if scores[0] < scores[1]:
            result = (0, 1)
        for i in range(2):
            strat[i]['wins'] += result[i]
            strat[i]['score'] += scores[i]
            strat[i]['games'] += 1
    print('\n\n================ total games: ' + str(n_games))
    for s in strategies:
        print('\n strategy ' + s['name'])
        if s['games']:
            plays = ' played against '
            for s_op in strategies:
                if s_op['name'] in s['plays']:
                    plays += s_op['name'] + ' (' + str(s['plays'][s_op['name']]) + ') '
            print('\t' + plays)
            print( 
              '\t played '+str(s['games'])+' times and won '+str(s['wins'])+' times with a global score of '+str(s['score']) +
                  '\n\t score/games: '+str(round(float(s['score'])/s['games'], 2))+ 
                  '\t wins/games: '+str(round(float(s['wins'])/s['games'], 2))+
                  '\t score/wins: '+(str(round(float(s['score'])/s['wins'], 2)) if s['wins'] else "--"))
        else:
            print("\t played no games.")
        
        
# tournament(50, 10, strategies, True) # test, with Verbose
tournament(500, 100, strategies) # short
tournament(5000, 100, strategies) # long



================ total games: 500

 strategy All-D
	 played against All-D (32) Random (30) Tit-For-Tat (24) Joss (37) Tester (28) MyStrategy (19) 
	 played 170 times and won 89 times with a global score of 38611
	 score/games: 227.12	 wins/games: 0.52	 score/wins: 433.83

 strategy Random
	 played against All-D (30) Random (20) Tit-For-Tat (34) Joss (35) Tester (28) MyStrategy (33) 
	 played 180 times and won 51 times with a global score of 40351
	 score/games: 224.17	 wins/games: 0.28	 score/wins: 791.2

 strategy Tit-For-Tat
	 played against All-D (24) Random (34) Tit-For-Tat (20) Joss (30) Tester (22) MyStrategy (28) 
	 played 158 times and won 0 times with a global score of 38915
	 score/games: 246.3	 wins/games: 0.0	 score/wins: --

 strategy Joss
	 played against All-D (37) Random (35) Tit-For-Tat (30) Joss (30) Tester (31) MyStrategy (23) 
	 played 186 times and won 117 times with a global score of 43509
	 score/games: 233.92	 wins/games: 0.63	 score/wins: 371.87

 strategy Te